In [1]:
import requests
import json
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [2]:
load_dotenv()

True

In [3]:
def get_viator_destinations():
    """
    Call the Viator destinations endpoint
        
    Returns:
        dict: JSON response from the Viator API
    """
    # API endpoint
    url = "https://api.viator.com/partner/destinations"
    
    # Headers
    headers = {
        "Accept": "application/json;version=2.0",
        "Accept-Language": "en-US",
        "exp-api-key": os.getenv("VIATOR_API_KEY")
    }
    
    # Make the request
    response = requests.get(url, headers=headers) #, params=params)
    
    # Check if request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

In [4]:
destinations = get_viator_destinations()
# destinations

In [5]:
destinations['totalCount']

3350

In [28]:
def get_viator_tags_en():
    """
    Call the Viator destinations endpoint
        
    Returns:
        dict: JSON response from the Viator API
    """
    try:
        # API endpoint
        url = "https://api.viator.com/partner/products/tags"
        
        # Headers
        headers = {
            "Accept": "application/json;version=2.0",
            "Accept-Language": "en-US",
            "exp-api-key": os.getenv("VIATOR_API_KEY")
        }
        
        # Make the request
        response = requests.get(url, headers=headers)
        
        # Check if request was successful
        if response.status_code == 200:
            tags = response.json()
            tags_en = {
                tag['tagId']: tag['allNamesByLocale']['en']
                for tag in tags['tags']
            }
            return tags_en
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            return None
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch destinations: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error fetching destinations: {e}")
        return None

In [31]:
tags_en = get_viator_tags_en()
# tags_en

In [32]:
# TODO: add pagination when more than 50 products.
def get_viator_products(destination_id):
    """
    Call the Viator destinations endpoint
        
    Returns:
        dict: JSON response from the Viator API
    """
    # API endpoint
    url = "https://api.viator.com/partner/products/search"
    
    # Headers
    headers = {
        "Accept": "application/json;version=2.0",
        "Accept-Language": "en-US",
        "exp-api-key": os.getenv("VIATOR_API_KEY"),
    }

    payload = {
        "filtering": {
            "destination": str(destination_id)
        },
        "currency": "USD",
        "pagination": {
            "count": 50 # 50 max
        }
    }
    
    # Make the request
    response = requests.post(url, headers=headers, json=payload)
    
    # Check if request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

In [33]:
location_name = 'Aspen'

aspen_locations = [
    d
    for d in destinations['destinations']
    if location_name.lower() in d['name'].lower()
]
assert 1 == len(aspen_locations)
print(aspen_locations)
aspen_location_id = aspen_locations[0]['destinationId']
aspen_location_id

[{'destinationId': 26395, 'name': 'Aspen', 'type': 'CITY', 'parentDestinationId': 273, 'lookupId': '8.77.273.26395', 'destinationUrl': 'https://www.viator.com/x/d26395-ttd?mcid=42383&pid=P00247797&medium=api&api_version=2.0', 'defaultCurrencyCode': 'USD', 'timeZone': 'America/Denver', 'iataCodes': ['ASE'], 'countryCallingCode': '+1', 'languages': ['en-US'], 'center': {'latitude': 39.191098, 'longitude': -106.817539}}]


26395

In [34]:
products = get_viator_products(aspen_location_id)
# products

{'products': [{'productCode': '74828P1',
   'title': "Aspen's DarkSide Ghost Tour",
   'description': "Enjoy an entertaining evening stroll through historic downtown as your guide uncovers Aspen's tarnished past. \xa0Hear the true tales from 1879 to today\xa0of Aspen's Ghosts, Murder & Mayhem.\n",
   'images': [{'imageSource': 'SUPPLIER_PROVIDED',
     'caption': "Aspen's DarkSide Guide",
     'isCover': True,
     'variants': [{'height': 50,
       'width': 50,
       'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-50x50/06/74/7c/8b.jpg'},
      {'height': 100,
       'width': 100,
       'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-100x100/06/74/7c/8b.jpg'},
      {'height': 200,
       'width': 200,
       'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-200x200/06/74/7c/8b.jpg'},
      {'height': 400,
       'width': 400,
       'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-400x400/06/74

In [39]:
[
    {
        **p,
        'tags_str': [tags_en[tag] for tag in p['tags']],
    }
    for p in products['products']
]

[{'productCode': '74828P1',
  'title': "Aspen's DarkSide Ghost Tour",
  'description': "Enjoy an entertaining evening stroll through historic downtown as your guide uncovers Aspen's tarnished past. \xa0Hear the true tales from 1879 to today\xa0of Aspen's Ghosts, Murder & Mayhem.\n",
  'images': [{'imageSource': 'SUPPLIER_PROVIDED',
    'caption': "Aspen's DarkSide Guide",
    'isCover': True,
    'variants': [{'height': 50,
      'width': 50,
      'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-50x50/06/74/7c/8b.jpg'},
     {'height': 100,
      'width': 100,
      'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-100x100/06/74/7c/8b.jpg'},
     {'height': 200,
      'width': 200,
      'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-200x200/06/74/7c/8b.jpg'},
     {'height': 400,
      'width': 400,
      'url': 'https://media-cdn.tripadvisor.com/media/attractions-splice-spp-400x400/06/74/7c/8b.jpg'},
     {'height': 

In [16]:
len(products['products'])

13

In [11]:
data_type = 'viator_product'
limit = None
products_data = products['products']

In [12]:
collections_data = {
    'viator_product': {
        "collection_name": "viator-products",
    },
}

print(f"Loading data for {data_type} into {collections_data[data_type]['collection_name']}")
collection_data = collections_data[data_type]

if limit:
    print(f"Limiting to a maximum of {limit} IDs")

# Get MongoDB credentials from environment variables
username = os.getenv("MONGODB_USERNAME")
password = os.getenv("MONGODB_PASSWORD")
cluster = os.getenv("MONGODB_CLUSTER")

# Construct MongoDB URI
uri = f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority&appName=Viammo-Cluster-alpha"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
client.admin.command('ping')
print("Pinged your deployment. You successfully connected to MongoDB!\n")

db = client["viammo-alpha"]
collection_name = collection_data["collection_name"]
collection = db[collection_name]

products_data = products_data[:limit] if limit else products_data
for product in products_data:
  productCode = product['productCode']

  # upsert into mongo
  result = collection.update_one(
    {"productCode": productCode},
    {"$set": product},
    upsert=True
  )
  print(f"Result for productCode {productCode}: {result}\n")

client.close()


Loading data for viator_product into viator-products
Pinged your deployment. You successfully connected to MongoDB!

Result for productCode 74828P1: UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002cf'), 'opTime': {'ts': Timestamp(1744496561, 1), 't': 719}, 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1744496561, 1), 'signature': {'hash': b'\x9d\xbe\xb3\x9d)w\xbeH\x1f\xb7_\xaa\x8f\x92\xb0\xfd\xb0\x8c$B', 'keyId': 7434008547316203532}}, 'operationTime': Timestamp(1744496561, 1), 'updatedExisting': True}, acknowledged=True)

Result for productCode 147508P175: UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002cf'), 'opTime': {'ts': Timestamp(1744496561, 6), 't': 719}, 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1744496561, 6), 'signature': {'hash': b'\x9d\xbe\xb3\x9d)w\xbeH\x1f\xb7_\xaa\x8f\x92\xb0\xfd\xb0\x8c$B', 'keyId': 7434008547316203532}}, 'operationTime': Timestamp(1744496561, 6), 'updatedExisting